# Boxplots

## Script setup

In [1]:
# Third-party and standard modules
from typing import Dict, Hashable, Union, Tuple, Optional
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import pytz
from collections import defaultdict
from numpy import NINF
import datetime
from importlib import reload
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
# User-defined modules.
from modules import dataframe_manip as dfm
from modules import misc, schemas, College1AcPtTimeSeries
# The reload() function is used to update modules after
# changes are made to their files.
dfm = reload(dfm)
misc = reload(misc)
schemas = reload(schemas)
College1AcPtTimeSeries = reload(College1AcPtTimeSeries)

In [2]:
schema: schemas.AcPtTimeSeries =\
    College1AcPtTimeSeries.College1AcPtTimeSeries

## Script parameters

The following values are this script's parameters.

1. filepath - path to csv file you would like to analyze
2. buildings - the list of buildings you would like to analyze
3. start_date and end_date - can be "None" if you wish to analyze the whole dataset
3. boxplot_hourly_average - set to true or false, depending if you want to display the total, weekday, and weekend graphs
4. holiday_analysis - set to true or false, if you would like to see the average occupancy of the holidays in the given date ranges

In [3]:
#filepath = './cisco_wifi_data_until_20190513.csv'
filepath = './wifi_data_until_20190204.csv'
timezone = pytz.timezone('US/Pacific')
buildings = ['SCC', 'HAHN', 'CROOKSHANK', 'THATHCHER', 'CLARK',
    'CARNEGIE', 'MASON', 'PEARSON', 'ALEXANDER', 'ITB']

start_date: Optional[str] = None  # '2018-09-04' #None
end_date: Optional[str] = None  #'2018-12-19' #None

boxplot_hourly_average = True
holiday_analysis = True

## Data Processing

In [4]:
# A time-series dataframe holding occupancy counts
# for many access points (columns).
occ: pd.DataFrame = dfm.csv_to_timeseries_df(
    filepath=filepath, timezone=timezone
)

In [5]:
# Convert to df values to strings in order to compare to given range
csv_start_date = occ.index[0].strftime('%Y-%m-%d')
csv_end_date = occ.index[-1].strftime('%Y-%m-%d')

In [6]:
# Date range error checking
if start_date is not None and end_date is not None:
    if end_date < start_date:
        print("end_date is before start_date")
        exit()

    if start_date < csv_start_date :
        print("Desired start_date is before the first day of the csv, please fix this to continue")
        exit()

    if end_date > csv_end_date:
        print("Desired end_date filter is after the last day of the csv, please fix this to continue")
        exit()

In [7]:
# Apply date range filter on the dataframe
if start_date is not None: 
    if end_date is None:
        occ = occ.loc[start_date:]
    else:
        occ = occ.loc[start_date:end_date]
else:
    if end_date is not None:
        occ = occ.loc[:end_date]

In [8]:
# Eliminate columns for access points not in the buildings we want.
occ = occ.drop(columns=list(filter(
    # add to drop-list if NOT in the buildings we want.
    lambda col: schema.col_to_building(col, safe=True) not in buildings,
    occ.columns.to_list()
)))

# We must assume that intervening na-values are 0's.
occ = dfm.fill_intervening_nas(
    occ, inplace=True, fill_val=0
)

In [9]:
occ_bybuild = occ.groupby(
    by=schema.col_to_building, axis=1
)

In [10]:
print('buildings not found: {}'.format(set(buildings) - set(occ_bybuild.groups.keys())))

# We can only work with the buildings we found.
buildings = list(occ_bybuild.groups.keys())

buildings not found: {'MASON', 'SCC', 'THATHCHER', 'HAHN', 'ALEXANDER', 'PEARSON', 'CARNEGIE', 'CROOKSHANK', 'ITB', 'CLARK'}


## Boxplots

In [11]:
if(boxplot_hourly_average == True):
    means_by_building: Dict[Hashable, pd.DataFrame] = {}
    maxmean = float('-inf')
    for build in buildings:
        means_by_building[build] = dfm.row_means(occ_bybuild.get_group(build))
        newmax = means_by_building[build].max()
        newmin = means_by_building[build].min()
        maxmean = newmax if newmax > maxmean else maxmean
    #print(maxmean)

    for build in buildings:
        mean_occ: pd.DataFrame = dfm.row_means(occ_bybuild.get_group(build))

        # Prints graph for average occupancy per hour of each each building 
        fig, ax = plt.subplots(figsize=(24, 15))
        ax.set_title(build)
        ax.set_xlabel('hour of the day')
        ax.set_ylabel('mean number of connected devices')
        ax.set_ylim(0, 200)

        seaborn.boxplot(
            # The hours (a number [0,23]) for each row.
            x=mean_occ.index.hour,
            # The total occupancy at that hour.
            y=mean_occ,
            ax=ax
        )

        # Prints graph of weekday average occupancy per hour of each each building 
        try:
            fig, ax = plt.subplots(figsize=(24, 15))
            ax.set_title(build + ' (weekdays)')
            ax.set_xlabel('hour of the day')
            ax.set_ylabel('mean number of connected devices')
            ax.set_ylim(0, maxmean)

            wkdays = mean_occ.loc[mean_occ.index.dayofweek<5].copy()
        
            seaborn.boxplot(
                # The hours (a number [0,23]) for each row.
                x=wkdays.index.hour,
                # The total occupancy at that hour.
                y=wkdays,
                ax=ax
            )
        except Exception as e:
            print("No weekdays dates available in data set. Error: {}".format(e))
            raise e
        
        
        # Prints graph of weekend average occupancy per hour of each each building 
        try:
            fig, ax = plt.subplots(figsize=(24, 15))
            ax.set_title(build + ' (weekends)')
            ax.set_xlabel('hour of the day')
            ax.set_ylabel('mean number of connected devices')
            ax.set_ylim(0, maxmean)

            wkends = mean_occ.loc[mean_occ.index.dayofweek>=5].copy()


            seaborn.boxplot(
                # The hours (a number [0,23]) for each row.
                x=wkends.index.hour,
                # The total occupancy at that hour.
                y=wkends,
                ax=ax
            )
        except Exception as e:
            print("No weekend dates available in data set. Error: {}".format(e))
            raise e
            
        
        # Saves the graphs as .png
        #fig.savefig('./{}.png'.format(build))

## Holidays

In [12]:
if (holiday_analysis == True):
    
    # Get all US holidays between time range
    holi_caldr = calendar()
    holidays = holi_caldr.holidays(start = start_date, end = end_date)

    # Shows what holidays are going to be included in this data set analysis 
    display(holidays)
    
    # Convert to date time inorder to compare with our df index
    holiday_ts = pd.to_datetime(holidays)

    truncated_time = occ.index.normalize()
    occ['time_no_timezone'] = truncated_time.tz_localize(None)
    
    # Check if the date matches our list of holidays
    occ['isHoliday'] = occ['time_no_timezone'].isin(holiday_ts)

    display(occ)

DatetimeIndex(['1970-01-01', '1970-02-16', '1970-05-25', '1970-07-03',
               '1970-09-07', '1970-10-12', '1970-11-11', '1970-11-26',
               '1970-12-25', '1971-01-01',
               ...
               '2030-01-01', '2030-01-21', '2030-02-18', '2030-05-27',
               '2030-07-04', '2030-09-02', '2030-10-14', '2030-11-11',
               '2030-11-28', '2030-12-25'],
              dtype='datetime64[ns]', length=594, freq=None)

,time_no_timezone,isHoliday
time,,
2018-07-18 16:10:14-07:00,2018-07-18,False
2018-07-18 16:15:23-07:00,2018-07-18,False
2018-07-18 16:20:38-07:00,2018-07-18,False
2018-07-18 16:25:06-07:00,2018-07-18,False
2018-07-18 16:30:06-07:00,2018-07-18,False
2018-07-18 16:35:08-07:00,2018-07-18,False
2018-07-18 16:40:12-07:00,2018-07-18,False
2018-07-18 16:45:16-07:00,2018-07-18,False
2018-07-18 16:50:19-07:00,2018-07-18,False


In [13]:
if (holiday_analysis == True):
    
    # Create a new df holiday_occ
    # If the date is a holiday, copy the data to holiday_occ
    holiday_occ = occ.loc[occ['isHoliday'] == True]
    #display(occ)
    holiday_occ = holiday_occ.drop(['time_no_timezone', 'isHoliday'], axis =1)
    occ_holiday_bybuild = holiday_occ.groupby(
        by=schema.col_to_building, axis=1
    )

    means_by_building: Dict[Hashable, pd.DataFrame] = {}
    maxmean = float('-inf')
    for build in buildings:
        means_by_building[build] = dfm.row_means(occ_holiday_bybuild.get_group(build))
        newmax = means_by_building[build].max()
        newmin = means_by_building[build].min()
        maxmean = newmax if newmax > maxmean else maxmean

    for build in buildings:
        
         # Prints graph of holiday average occupancy per hour of each each building 
        try:
            mean_occ_holiday: pd.DataFrame = dfm.row_means(occ_holiday_bybuild.get_group(build))

            fig, ax = plt.subplots(figsize=(24, 15))
            ax.set_title(build + ' (holidays from ' + start_date + ' to ' + end_date +')')
            ax.set_xlabel('hour of the day')
            ax.set_ylabel('mean number of connected devices')
            ax.set_ylim(0, 200)

            seaborn.boxplot(
                # The hours (a number [0,23]) for each row.
                x=mean_occ_holiday.index.hour,
                # The total occupancy at that hour.
                y=mean_occ_holiday,
                ax=ax
            )
        except Exception as e:
            print("No holidays in this data set. Error: {}".format(e))
            raise e